In [ ]:
import numpy as np 
import pandas as pd

In [ ]:
#dataset loading 
from keras.datasets import fashion_mnist
#global x_train
#global y_train
#global x_test
#global y_test
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train=x_train.reshape(x_train.shape[0],(x_train.shape[1]*x_train.shape[2]))
x_test=x_test.reshape(x_test.shape[0],(x_test.shape[1]*x_test.shape[2]))
x_train=x_train/255
x_test=x_test/255


4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
#ab chalu karte hain kuch main kaam kaaj
#apne ko 3 cheez ki zarurat padegi 
#weight store karne ke liye ek 3 dimensional matrix(l,m,n)
#bias store krane ke liye matrix(l*bias)
#activation store krane ke liye list chaiye of matrix(l,m,n)
#postactivation store karwane ke liye phir chaiye rhega
#when i make a layer then it will give me the corresponding weight matrix, bias term 
#forward pass ka ek function likhna padega 
#backward pass ka alag function likhna pdega

In [ ]:
def sigmoid(z):
  #shape of x will be n*k w will be k*m so x*w will give us n*m and b will be 1*m so using vectorization we get n*m finally 
  #z=np.matmul(x,w) # dimension has become n*m
  #z=np.add(z,b)
  for i in range(z.shape[0]):
    idx=np.argmax(z[i])
    #idx2=np.argmin(z[i])
  #print(z)
  return 1/(1+np.exp(-z))

In [ ]:
def sigmoid_derivative(z):
  return np.multiply((1/(1+np.exp(-z))),(1-(1/(1+np.exp(-z)))))

In [ ]:
def softmax(x):
  #x=np.copy(z)
  for i in range(x.shape[0]):
    sum=0
    #minidx=np.argmin(x[i])
    #maxidx=np.argmax(x[i])
    #x[i]=(x[i]-x[i][minidx])/(x[i][maxidx]-x[i][minidx])
    for j in range(x.shape[1]):
      sum=sum+np.exp(x[i][j])
    x[i]=np.exp(x[i])/sum
  return x

In [ ]:
# def softmax_helper(z):
#     assert len(z.shape) == 2
#     s = np.max(z, axis=1)
#     s = s[:, np.newaxis] # necessary step to do broadcasting
#     e_x = np.exp(z - s)
#     div = np.sum(e_x, axis=1)
#     div = div[:, np.newaxis] # dito
#     return e_x / div

In [ ]:
def cross_entropy(yhat,y_train):
  #chalo game shuru karte hain
  #yhat n*10
  sum=0 
  for i in range(y_train.shape[0]):
    sum+=-((np.log2(yhat[i][y_train[i]])))
  return sum

In [ ]:
def make_layer(w,b,input,output):
  weights=np.random.uniform(-1,1,(input,output))
  bias=np.random.uniform(-1,1,(1,output))
  w.append(weights)
  b.append(bias)
  return (w,b)

In [ ]:
def forward_pass(x,w,b,a,h):
  l=len(w)
  #use a for loop for the required functionality
  for i in range (l-1):
    a1=np.add(np.matmul(x,w[i]),b[i])
    h1=sigmoid(a1)
    a.append(a1)
    h.append(h1)
    x=h1
  #this loop will complete the forward pass
  #for the last layer
  a1=np.add(np.matmul(x,w[l-1]),b[l-1])
  h1=softmax(a1)
  a.append(a1)
  h.append(h1)
  return (a,h)

In [ ]:
def backward_pass(wd,bd,ad,hd,w,b,a,h,yhat,y_train,x_train,no_of_classes):
  #yahan par derivative ulte calculate honge yeh yaad rakhna,to saare loop ulte chalane honge humko
  #apne ko yl banana padega for all the n inputs
  #last layer ke liye special treatment
  el=np.zeros((y_train.shape[0],no_of_classes))
  for i in range (y_train.shape[0]):
      el[i][y_train[i]]=1

  yhatl=np.zeros((yhat.shape[0],1))
  for i in range (yhat.shape[0]):
    yhatl[i]=yhat[i][y_train[i]]
  #calculate for the last layer
  hd1=-(el/yhatl)
  ad1=-(el-yhat)
  hd.append(hd1)
  ad.append(ad1)
  #ab loop me daal kar saare ke saare humesha last appended wala hi use hoga
  l=len(w)
  for i in range(l-1,-1,-1):
    # print("i "+str(i))
    # print(h[i-1].shape)
    #print(ad[len(ad)-1].shape)
    q=h[i-1].T
    if i==0:
      q=x_train.T
    wi=np.matmul(q,ad[len(ad)-1])/x_train.shape[0]
    bi=np.sum(ad[len(ad)-1],axis=0)/x_train.shape[0]
    if i!=0:
      hd1=np.matmul(ad[len(ad)-1],w[i].T)
      sig=sigmoid_derivative(a[i-1])
      ad1=np.multiply(hd1,sig)
      hd.append(hd1)
      ad.append(ad1)
    wd.append(wi)
    bd.append(bi)
  return (wd,bd)  

In [ ]:
def test(x_test,y_test,w,b):
  a=[]
  h=[]
  a,h=forward_pass(x_test,w,b,a,h)
  l=len(w)
  ypred=np.argmax(h[l-1],axis=1)
  #print(ypred)
  #print(y_test)
  count=0
  for i in range(y_test.shape[0]):
    if ypred[i]!=y_test[i]:
      count=count+1
  print((x_test.shape[0]-count)/y_test.shape[0])


In [ ]:
def gradientDescent(x_train,y_train,x_test,y_test,no_of_classes,w,b,l,iter,n):
  for i in range (iter):
    a=[]
    h=[]
    wd=[]
    bd=[]
    ad=[]
    hd=[]
    a,h=forward_pass(x_train,w,b,a,h)
    #print(a[0][:])
    #print(a[l-1][:])
    loss=cross_entropy(h[l-1],y_train)
    print("Iteration Number: "+str(i)+" loss: "+str(loss/x_train.shape[0]))
    wd,bd=backward_pass(wd,bd,ad,hd,w,b,a,h,h[l-1],y_train,x_train)
    for j in range (l):
      w[j]=w[j]-n*wd[l-1-j]
      b[j]=b[j]-n*bd[l-1-j]
  

  print("Test Results: ")
  test(x_test,y_test,w,b)




In [ ]:
def oneGradientDescent(x_train,y_train,x_test,y_test,no_of_classes,w,b,l,n):
  a=[]
  h=[]
  wd=[]
  bd=[]
  ad=[]
  hd=[]
  a,h=forward_pass(x_train,w,b,a,h)
    
  #loss=cross_entropy(h[l-1],y_train)
  # print(" loss: "+str(loss/x_train.shape[0]))
  wd,bd=backward_pass(wd,bd,ad,hd,w,b,a,h,h[l-1],y_train,x_train,no_of_classes)
  for j in range (l):
    w[j]=w[j]-n*wd[l-1-j]
    b[j]=b[j]-n*bd[l-1-j]
  

  # print("Test Results: ")
  # test(x_test,y_test,w,b)
  return (w,b)




In [ ]:
def batchGrad(x_train,y_train,x_test,y_test,no_of_classes,w,b,l,iter,n,batchSize):
  data=[]
  ans=[]
  for i in range(int(x_train.shape[0]/batchSize)):
    batch=[]
    batch_ans=[]
    for j in range(i*batchSize,min((i+1)*batchSize,x_train.shape[0]),1):
      batch.append(x_train[j])
      batch_ans.append(y_train[j])
    batch=np.array(batch)
    batch_ans=np.array(batch_ans)
    data.append(batch)
    ans.append(batch_ans)
  
  #start the batch gradient descent 
  for i in range(iter):
    h=None
    for j in range(len(data)):
      w,b=oneGradientDescent(data[j],ans[j],x_test,y_test,no_of_classes,w,b,l,n)
    a=[]
    h=[]
    a,h=forward_pass(x_train,w,b,a,h)
    loss=cross_entropy(h[l-1],y_train)
    print("Iteration Number: "+str(i)+" loss: "+str(loss/x_train.shape[0]))
    print("Test Results: ")
    test(x_test,y_test,w,b) 



In [ ]:
def momentumGradientDescent(x_train,y_train,x_test,y_test,no_of_classes,w,b,l,iter,n,batchSize,beta):
  data=[]
  ans=[]
  for i in range(int(x_train.shape[0]/batchSize)):
    batch=[]
    batch_ans=[]
    for j in range(i*batchSize,min((i+1)*batchSize,x_train.shape[0]),1):
      batch.append(x_train[j])
      batch_ans.append(y_train[j])
    batch=np.array(batch)
    batch_ans=np.array(batch_ans)
    data.append(batch)
    ans.append(batch_ans)
  #batches have been made accordingly
  moment=[]
  momentB=[]
  for i in range(l):
    temp=np.zeros((w[i].shape))
    temp2=np.zeros(b[i].shape)
    moment.append(temp)
    momentB.append(temp2)
  print(len(data))
  for i in range(iter):
    h=None
    for j in range(len(data)):
      a=[]
      h=[]
      wd=[]
      bd=[]
      ad=[]
      hd=[]
      if j%10==0:
        print("Batch Number "+str(j))
      a,h=forward_pass(x_train,w,b,a,h)
    
      #loss=cross_entropy(h[l-1],y_train)
      # print(" loss: "+str(loss/x_train.shape[0]))
      wd,bd=backward_pass(wd,bd,ad,hd,w,b,a,h,h[l-1],y_train,x_train,no_of_classes)
      for k in range (l):
        moment[k]=(moment[k]*beta)+wd[l-1-k]
        momentB[k]=(momentB[k]*beta)+bd[l-1-k]
        w[k]=w[k]-n*moment[k]
        b[k]=b[k]-n*momentB[k]
      
    a=[]
    h=[]
    a,h=forward_pass(x_train,w,b,a,h)
    loss=cross_entropy(h[l-1],y_train)
    print("Iteration Number: "+str(i)+" loss: "+str(loss/x_train.shape[0]))
    print("Test Results: ")
    test(x_test,y_test,w,b) 




In [ ]:
def nsGradientDescent(x_train,y_train,x_test,y_test,no_of_classes,w,b,l,iter,n,batchSize,beta):
  data=[]
  ans=[]
  for i in range(int(x_train.shape[0]/batchSize)):
    batch=[]
    batch_ans=[]
    for j in range(i*batchSize,min((i+1)*batchSize,x_train.shape[0]),1):
      batch.append(x_train[j])
      batch_ans.append(y_train[j])
    batch=np.array(batch)
    batch_ans=np.array(batch_ans)
    data.append(batch)
    ans.append(batch_ans)
  #batches have been made accordingly
  moment=[]
  momentB=[]
  for i in range(l):
    temp=np.zeros((w[i].shape))
    temp2=np.zeros((b[i].shape))
    moment.append(temp)
    momentB.append(temp2)
  print(len(data))
  for i in range(iter):
    h=None
    for j in range(len(data)):
      a=[]
      h=[]
      wd=[]
      bd=[]
      ad=[]
      hd=[]
      if j%10==0:
        print("Batch Number "+str(j))
      
    
      #loss=cross_entropy(h[l-1],y_train)
      # print(" loss: "+str(loss/x_train.shape[0]))
      for k in range(l):
        old=moment[k]*beta
        oldB=momentB[k]*beta
        w[k]=w[k]-old
        b[k]=b[k]-oldB
      a,h=forward_pass(x_train,w,b,a,h)  
      wd,bd=backward_pass(wd,bd,ad,hd,w,b,a,h,h[l-1],y_train,x_train,no_of_classes)
      for k in range (l):
        old=moment[k]*beta
        oldB=momentB[k]*beta
        moment[k]=(old)+wd[l-1-k]
        momentB[k]=oldB+bd[l-1-k]
        w[k]=w[k]-n*moment[k]
        b[k]=b[k]-n*momentB[k]
      
    a=[]
    h=[]
    a,h=forward_pass(x_train,w,b,a,h)
    loss=cross_entropy(h[l-1],y_train)
    print("Iteration Number: "+str(i)+" loss: "+str(loss/x_train.shape[0]))
    print("Test Results: Nestorov Gradient Descent :: ")
    test(x_test,y_test,w,b) 

In [ ]:
def architecture(x_train,y_train,x_test,y_test,no_of_classes):

  w=[]
  b=[]
  w,b=make_layer(w,b,784,128)
  w,b=make_layer(w,b,128,128)
  w,b=make_layer(w,b,128,128)
  w,b=make_layer(w,b,128,10)
  l=len(w)
  iter=2
  n=0.1
  #gradientDescent(x_train,y_train,x_test,y_test,no_of_classes,w,b,l,iter,n)
  print("Batch Gradient Descent: ")
  batchGrad(x_train,y_train,x_test,y_test,no_of_classes,w,b,l,iter,n,1024)
  print("Momentum Gradient Descent: ")
  momentumGradientDescent(x_train,y_train,x_test,y_test,no_of_classes,w,b,l,iter,n,1024,0.9)
  print("Starting of Nestrov Gradient Descent: ")
  nsGradientDescent(x_train,y_train,x_test,y_test,no_of_classes,w,b,l,iter,n,1024,0.9)
  

In [ ]:
architecture(x_train,y_train,x_test,y_test,10)

Batch Gradient Descent: 
Iteration Number: 0 loss: 1.8360265145387435
Test Results: 
0.5781
Iteration Number: 1 loss: 1.4435367844432228
Test Results: 
0.6633
Momentum Gradient Descent: 
58
Batch Number 0
Batch Number 10
Batch Number 20
Batch Number 30
Batch Number 40
Batch Number 50
Iteration Number: 0 loss: 0.9237465649702795
Test Results: 
0.7591
Batch Number 0
Batch Number 10
Batch Number 20
Batch Number 30
Batch Number 40
Batch Number 50
Iteration Number: 1 loss: 0.811037050019849
Test Results: 
0.7875
Starting of Nestrov Gradient Descent: 
58
Batch Number 0


ValueError: ignored

In [ ]:
def architecture(x_train,y_train,x_test,y_test,no_of_classes):

  w=[]
  b=[]
  w,b=make_layer(w,b,784,128)
  w,b=make_layer(w,b,128,128)
  w,b=make_layer(w,b,128,128)
  w,b=make_layer(w,b,128,10)
  l=len(w)
  iter=2
  n=0.1
  #gradientDescent(x_train,y_train,x_test,y_test,no_of_classes,w,b,l,iter,n)
  #print("Batch Gradient Descent: ")
  #batchGrad(x_train,y_train,x_test,y_test,no_of_classes,w,b,l,iter,n,1024)
  #print("Momentum Gradient Descent: ")
  #momentumGradientDescent(x_train,y_train,x_test,y_test,no_of_classes,w,b,l,iter,n,1024,0.9)
  print("Starting of Nestrov Gradient Descent: ")
  nsGradientDescent(x_train,y_train,x_test,y_test,no_of_classes,w,b,l,iter,n,1024,0.9)
  

In [ ]:
architecture(x_train,y_train,x_test,y_test,10)

Starting of Nestrov Gradient Descent: 
58
Batch Number 0
Batch Number 10
Batch Number 20
Batch Number 30
Batch Number 40
Batch Number 50
Iteration Number: 0 loss: 50.16974652624729
Test Results: Nestorov Gradient Descent :: 
0.1
Batch Number 0
Batch Number 10
Batch Number 20
Batch Number 30
Batch Number 40
Batch Number 50
Iteration Number: 1 loss: 34.613668183574966
Test Results: Nestorov Gradient Descent :: 
0.1
